In [15]:
# Importamos las librerías que necesitamos
# 
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest


# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Fase 1: Exploración y Limpieza de Customer Loyalty History

In [16]:
df_CLH = pd.read_csv('Customer Loyalty History.csv')
df_CLH.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [17]:
df_CLH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

Cambio las columnas de numérica a categórica en Loyalty Number, Enrollment Year, Enrollment Month ✔

In [18]:
df_CLH['Loyalty Number'] = df_CLH['Loyalty Number'].astype('object')
df_CLH['Loyalty Number'].dtype

dtype('O')

In [19]:
df_CLH['Enrollment Year'] = df_CLH['Enrollment Year'].astype('object')
df_CLH['Enrollment Year'].dtype

dtype('O')

In [20]:
df_CLH['Enrollment Month'] = df_CLH['Enrollment Month'].astype('object')
df_CLH['Enrollment Month'].dtype

dtype('O')

Estudio duplicados.

In [24]:
df_CLH.duplicated().sum()

0

Estudio nulos.

In [23]:
df_CLH.isnull().sum()[df_CLH.isnull().sum()>0]

Salary                 4238
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

Los valores nulos de Cancellation Month y Cancellation Year significan que el cliente todavía no ha cancelado y, por lo tanto, sigue en el programa. Los imputaremos con una nueva categoría: 'vigente'

In [10]:
df_CLH['Cancellation Month'] = df_CLH['Cancellation Month'].fillna('vigente')
df_CLH['Cancellation Month']

0        vigente
1        vigente
2            1.0
3        vigente
4        vigente
          ...   
16732    vigente
16733    vigente
16734    vigente
16735    vigente
16736        8.0
Name: Cancellation Month, Length: 16737, dtype: object

In [25]:
df_CLH['Cancellation Year'] = df_CLH['Cancellation Year'].fillna('vigente')
df_CLH['Cancellation Year']

0        vigente
1        vigente
2         2018.0
3        vigente
4        vigente
          ...   
16732    vigente
16733    vigente
16734    vigente
16735    vigente
16736     2016.0
Name: Cancellation Year, Length: 16737, dtype: object

Ahora que están en string, podría cambiar los meses en formato float a formato mes en palabra.

In [ ]:
def formato_mes (cadena):

    if cadena == 'vigente':
        resultado = cadena
    elif cadena == '1.0':
        resultado = 'enero'

In [12]:
df_CLH_copia = df_CLH.copy()

In [13]:
""" diccionario_meses = {
                        'vigente':'vigente',
                        '1.0' : 'enero',
                        '2.0' : 'febrero',
                        '3.0' : 'marzo',
                        '4.0' : 'abril',
                        '5.0' : 'mayo',
                        '6.0' : 'junio',
                        '7.0' : 'julio',
                        '8.0' : 'agosto',
                        '9.0' : 'septiembre',
                        '10.0' : 'octubre',
                        '11.0' : 'noviembre',
                        '12.0' : 'diciembre'
}

df_CLH['Nueva Cancellation Month'].map(diccionario_meses)
df_CLH['Cancellation Month'] """ 


" diccionario_meses = {\n                        'vigente':'vigente',\n                        '1.0' : 'enero',\n                        '2.0' : 'febrero',\n                        '3.0' : 'marzo',\n                        '4.0' : 'abril',\n                        '5.0' : 'mayo',\n                        '6.0' : 'junio',\n                        '7.0' : 'julio',\n                        '8.0' : 'agosto',\n                        '9.0' : 'septiembre',\n                        '10.0' : 'octubre',\n                        '11.0' : 'noviembre',\n                        '12.0' : 'diciembre'\n}\n\ndf_CLH['Nueva Cancellation Month'].map(diccionario_meses)\ndf_CLH['Cancellation Month'] "